In [ ]:
# if you have google drive permission then use this 

from google.colab import drive
drive.mount('/gdrive')
!ln -s   /gdrive/MyDrive/colab/Credit_Card/data/ /content/data

Mounted at /gdrive


# Start


In [ ]:
f1id = '1W0EL3w6qxHRa5ZFPYhYl87Piv7Y94ELG'
f2id = '1mJv80_a17wXnXydZEPGXKFtnQRd5REsc'

!pip3 install gdown
import gdown
url = 'https://drive.google.com/uc?id=%s'%(f1id)
output = 'train.csv'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?id=%s'%(f2id)
output = 'test.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1W0EL3w6qxHRa5ZFPYhYl87Piv7Y94ELG
To: /content/train.csv
99.2MB [00:00, 301MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mJv80_a17wXnXydZEPGXKFtnQRd5REsc
To: /content/test.csv
33.1MB [00:00, 258MB/s]


'test.csv'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
! mkdir data model submit

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘model’: File exists
mkdir: cannot create directory ‘submit’: File exists


In [ ]:
train = pd.read_csv('train.csv').set_index('txkey')
train.shape

(1141340, 22)

In [ ]:
train.head(2)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
txkey,,,,,,,,,,,,,,,,,,,,,,
516056,6881,113261,38038,513.8,5,0,N,0,N,N,0,5,N,0,33,172652.0,457,59333,N,0,102,0
483434,6881,15408,188328,513.8,5,0,N,0,N,N,0,5,N,0,6,152458.0,457,59333,N,0,102,0


In [ ]:
y_train = train['fraud_ind']
# df = df.loc[:, df.columns != 'fraud_ind']

In [ ]:
y_train
Fcnt = 0
Ncnt = 0
for val in y_train:
  if val == 1: Fcnt += 1
  else: Ncnt += 1

print('================== in Train Set ==================')
print(f'[Fraud] {Fcnt} cases')
print(f'[Normal] {Ncnt} cases')

================== in Train Set ==================
[Fraud] 15283 cases
[Normal] 1126057 cases


In [ ]:
# testing data 的要放進來，這樣 labeling encoding 才不會漏掉
test = pd.read_csv('test.csv').set_index('txkey')
y_test = test['fraud_ind']
test.shape

(380447, 22)

In [ ]:
test.head(2)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
txkey,,,,,,,,,,,,,,,,,,,,,,
4376,0,134508,45725,465.62,5,0,N,2,N,N,0,0,N,0,9,105114.0,451,0,N,5817,102,0
2943,0,78377,2295,465.62,5,0,N,2,N,N,0,0,N,0,6,104918.0,451,0,N,5817,102,0


In [ ]:
print('============ TEST SET ===============')
Fcnt = 0
Ncnt = 0
for val in y_test:
  if val == 1: Fcnt += 1
  else: Ncnt += 1

print(f'[Fraud] {Fcnt} cases')
print(f'[Normal] {Ncnt} cases')

============ TEST SET ===============
[Fraud] 5072 cases
[Normal] 375375 cases


In [ ]:
df = pd.concat((train, test), 0)
df.shape

(1521787, 22)

In [ ]:
y = pd.concat((y_train, y_test), 0)
y.shape

(1521787,)

In [ ]:
df.head(5)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
txkey,,,,,,,,,,,,,,,,,,,,,,
516056,6881,113261,38038,513.80,5,0,N,0,N,N,0,5,N,0,33,172652.0,457,59333,N,0,102,0
483434,6881,15408,188328,513.80,5,0,N,0,N,N,0,5,N,0,6,152458.0,457,59333,N,0,102,0
1407164,6716,157159,29967,1016.11,5,62,N,5,N,N,0,5,N,0,5,172946.0,247,50436,N,3281,102,0
1051004,5975,105985,81305,713.66,5,62,N,4,N,N,0,5,N,0,6,182129.0,263,93775,N,5817,102,0
1622153,6411,94435,49219,1806.49,3,62,N,4,N,N,0,5,N,0,6,172624.0,339,0,N,5865,102,0


# **欄位類別判定**

In [ ]:
# df.keys()
# Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
#        'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt', 'loctm',
#        'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd'],
#       dtype='object')

# col_names_cont = [] # 數值型資料
# col_names_disc = [] # 類別型資料
# col_has_na = [] # 待補NA資料
# for c in df.keys():
#     uni = df[c].unique()
#     n_na = pd.isna(df[c]).sum() # NA 數量
#     if n_na > 0:
#         # 發現NA資料
#         col_has_na.append(c)
#         print(f"{c} has NAN values...")   
#     if len(uni) < 200 :
#         # Unique數值小於200個 判定為 類別型資料
#         print(f"{c}: Unique Type={uni}")
#         col_names_disc.append(c)
#     else:
#         info = [ df[c].max(), df[c].min(), df[c].mean(), df[c].std()]
#         info = [ round(x,2) for x in info ]
#         offset = 1 if n_na else 0
#         diversity = (len(uni)-offset)/(len(df)-n_na)
#         print(f"{c}: dtype={df[c].dtype}, #NA={n_na}")
#         print("       max={}, min={}, mean={}, std={}, diversity={:.2f}%".format(*info, diversity*100 ) )
#         if diversity == 1.0:
#             print(f"       Delete col [{c}] due to diversity is 100% ")
#         else:
#             col_names_cont.append(c)
#     print('==================================================================')

# print(f'CONTINUOUS: {col_names_cont}')
# print(f'CATEGORY:{col_names_disc}')
# print(f'NA: {col_has_na}')

In [ ]:
drops = ['bacno', 'cano', 'locdt', 'loctm', 'csmcu', 'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'ovrlt']

col_names_cont = ['conam'] # 數值型資料
col_names_disc = ['contp', 'etymd', 'stocn', 'stscd'] # 類別型資料
already_binary = ['ecfg', 'fraud_ind']
manipulation = ['acqic', 'mcc', 'mchno', 'scity']
col_has_na = ['flbmk', 'flg_3dsmk'] # 待補NA資料

df

,acqic,ecfg,fraud_ind,mcc,mchno,scity,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0,mcc_289,mcc_380,mcc_192,mcc_288,mcc_203,mcc_251,mcc_247,mcc_263,mcc_457,mcc_292,mchno_55807,mchno_59333,mchno_45597,mchno_67588,mchno_66634,mchno_86584,mchno_82174,mchno_78638,mchno_59369,mchno_44859,mchno_544,scity_6580,scity_3460,scity_6593,scity_1337,scity_5320,conam_3400
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,6881,0,0,457,59333,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
483434,6881,0,0,457,59333,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1407164,6716,0,0,247,50436,3281,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1051004,5975,0,0,263,93775,5817,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1622153,6411,0,0,339,0,5865,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,5588,1,0,296,73998,3426,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1846986,6215,1,0,289,54828,3460,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
992751,6231,0,0,245,18405,5817,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = df.drop(df.loc[:,drops], axis = 1)

In [ ]:
df

,acqic,conam,contp,ecfg,etymd,fraud_ind,mcc,mchno,scity,stocn,stscd
txkey,,,,,,,,,,,
516056,6881,513.80,5,N,0,0,457,59333,0,102,0
483434,6881,513.80,5,N,0,0,457,59333,0,102,0
1407164,6716,1016.11,5,N,5,0,247,50436,3281,102,0
1051004,5975,713.66,5,N,4,0,263,93775,5817,102,0
1622153,6411,1806.49,3,N,4,0,339,0,5865,102,0
...,...,...,...,...,...,...,...,...,...,...,...
668463,5588,1.38,5,Y,8,0,296,73998,3426,42,0
1846986,6215,609.21,5,Y,2,0,289,54828,3460,46,0
992751,6231,574.36,5,N,5,0,245,18405,5817,102,0


In [ ]:
# 這些資料有缺值
print(col_has_na)

['flbmk', 'flg_3dsmk']


In [ ]:
# 處理缺失值 全部填"0"
df = df.fillna("NA")

# **特殊處理 (類別型)**

In [ ]:
key = 'acqic'
indexes = pd.value_counts(train[train.fraud_ind==1][key]).iloc[:10].index

for index in indexes:
  column_name = f'{key}_{index}'
  df[column_name] = np.where(df[key]== index, 1, 0)

# ==================================================================
indexes = pd.value_counts(train[train.fraud_ind==0][key]).iloc[:10].index
for index in indexes:
  column_name = f'{key}_{index}'
  df[column_name] = np.where(df[key]== index, 1, 0)

# x['acqic_3348'].describe()
df

,acqic,conam,contp,ecfg,etymd,fraud_ind,mcc,mchno,scity,stocn,stscd,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,6881,513.80,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
483434,6881,513.80,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1407164,6716,1016.11,5,N,5,0,247,50436,3281,102,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1051004,5975,713.66,5,N,4,0,263,93775,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1622153,6411,1806.49,3,N,4,0,339,0,5865,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,5588,1.38,5,Y,8,0,296,73998,3426,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1846986,6215,609.21,5,Y,2,0,289,54828,3460,46,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
992751,6231,574.36,5,N,5,0,245,18405,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
key = 'mcc'
max_cnt = 5
indexes = pd.value_counts(train[train.fraud_ind==1][key]).iloc[:max_cnt].index

for index in indexes:
  column_name = f'{key}_{index}'
  df[column_name] = np.where(df[key]== index, 1, 0)

# ==================================================================
indexes = pd.value_counts(train[train.fraud_ind==0][key]).iloc[:max_cnt].index
for index in indexes:
  column_name = f'{key}_{index}'
  df[column_name] = np.where(df[key]== index, 1, 0)

df

,acqic,conam,contp,ecfg,etymd,fraud_ind,mcc,mchno,scity,stocn,stscd,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0,mcc_289,mcc_380,mcc_192,mcc_288,mcc_203,mcc_251,mcc_247,mcc_263,mcc_457,mcc_292
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,6881,513.80,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
483434,6881,513.80,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1407164,6716,1016.11,5,N,5,0,247,50436,3281,102,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1051004,5975,713.66,5,N,4,0,263,93775,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1622153,6411,1806.49,3,N,4,0,339,0,5865,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,5588,1.38,5,Y,8,0,296,73998,3426,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1846986,6215,609.21,5,Y,2,0,289,54828,3460,46,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
992751,6231,574.36,5,N,5,0,245,18405,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
key = 'mchno'
max_cnt = 6
indexes = [55807, 59333, 45597, 67588, 66634]

for index in indexes:
  column_name = f'{key}_{index}'
  df[column_name] = np.where(df[key]== index, 1, 0)

# ==================================================================
indexes = [86584, 82174, 78638, 59369, 44859, 544]
for index in indexes:
  column_name = f'{key}_{index}'
  df[column_name] = np.where(df[key]== index, 1, 0)

df

,acqic,conam,contp,ecfg,etymd,fraud_ind,mcc,mchno,scity,stocn,stscd,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0,mcc_289,mcc_380,mcc_192,mcc_288,mcc_203,mcc_251,mcc_247,mcc_263,mcc_457,mcc_292,mchno_55807,mchno_59333,mchno_45597,mchno_67588,mchno_66634,mchno_86584,mchno_82174,mchno_78638,mchno_59369,mchno_44859,mchno_544
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,6881,513.80,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
483434,6881,513.80,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1407164,6716,1016.11,5,N,5,0,247,50436,3281,102,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1051004,5975,713.66,5,N,4,0,263,93775,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1622153,6411,1806.49,3,N,4,0,339,0,5865,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,5588,1.38,5,Y,8,0,296,73998,3426,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1846986,6215,609.21,5,Y,2,0,289,54828,3460,46,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
992751,6231,574.36,5,N,5,0,245,18405,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
key = 'scity'
max_cnt = 6
indexes = [6580, 3460, 6593, 1337, 5320]

for index in indexes:
  column_name = f'{key}_{index}'
  df[column_name] = np.where(df[key]== index, 1, 0)

df

,acqic,conam,contp,ecfg,etymd,fraud_ind,mcc,mchno,scity,stocn,stscd,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0,mcc_289,mcc_380,mcc_192,mcc_288,mcc_203,mcc_251,mcc_247,mcc_263,mcc_457,mcc_292,mchno_55807,mchno_59333,mchno_45597,mchno_67588,mchno_66634,mchno_86584,mchno_82174,mchno_78638,mchno_59369,mchno_44859,mchno_544,scity_6580,scity_3460,scity_6593,scity_1337,scity_5320
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,6881,513.80,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
483434,6881,513.80,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1407164,6716,1016.11,5,N,5,0,247,50436,3281,102,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1051004,5975,713.66,5,N,4,0,263,93775,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1622153,6411,1806.49,3,N,4,0,339,0,5865,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,5588,1.38,5,Y,8,0,296,73998,3426,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1846986,6215,609.21,5,Y,2,0,289,54828,3460,46,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
992751,6231,574.36,5,N,5,0,245,18405,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **數值型 資料處理**

In [ ]:
df_cont = df[col_names_cont].copy()  # 只拿 數值型 的資料們
df_cont.head()  # 標準化之前

,conam
txkey,
516056,513.80
483434,513.80
1407164,1016.11
1051004,713.66
1622153,1806.49


In [ ]:
# if conam exceeds NT$3400: 1
def f(row):
    if row['conam'] < 3400:
        val = 0
    elif row['conam'] >= 3400:
        val = 1
    return val

df['conam_3400'] = df.apply(f, axis=1)
df

,acqic,conam,contp,ecfg,etymd,fraud_ind,mcc,mchno,scity,stocn,stscd,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0,mcc_289,mcc_380,mcc_192,mcc_288,mcc_203,mcc_251,mcc_247,mcc_263,mcc_457,mcc_292,mchno_55807,mchno_59333,mchno_45597,mchno_67588,mchno_66634,mchno_86584,mchno_82174,mchno_78638,mchno_59369,mchno_44859,mchno_544,scity_6580,scity_3460,scity_6593,scity_1337,scity_5320,conam_3400
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,6881,513.80,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
483434,6881,513.80,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1407164,6716,1016.11,5,N,5,0,247,50436,3281,102,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1051004,5975,713.66,5,N,4,0,263,93775,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1622153,6411,1806.49,3,N,4,0,339,0,5865,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,5588,1.38,5,Y,8,0,296,73998,3426,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1846986,6215,609.21,5,Y,2,0,289,54828,3460,46,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
992751,6231,574.36,5,N,5,0,245,18405,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df['conam_3400'].describe()

count    1.521787e+06
mean     3.541889e-04
std      1.881658e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: conam_3400, dtype: float64

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_cont = scaler.fit_transform(df_cont)

In [ ]:
df_cont = pd.DataFrame(data=X_cont, index=df.index, columns=col_names_cont)
df_cont.head()  # 標準化之後的資料

,conam
txkey,
516056,-0.349849
483434,-0.349849
1407164,0.897173
1051004,0.146318
1622153,2.859350


In [ ]:
df = df.drop(df.loc[:,['conam']], axis = 1)

In [ ]:
df

,acqic,contp,ecfg,etymd,fraud_ind,mcc,mchno,scity,stocn,stscd,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0,mcc_289,mcc_380,mcc_192,mcc_288,mcc_203,mcc_251,mcc_247,mcc_263,mcc_457,mcc_292,mchno_55807,mchno_59333,mchno_45597,mchno_67588,mchno_66634,mchno_86584,mchno_82174,mchno_78638,mchno_59369,mchno_44859,mchno_544,scity_6580,scity_3460,scity_6593,scity_1337,scity_5320,conam_3400
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,6881,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
483434,6881,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1407164,6716,5,N,5,0,247,50436,3281,102,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1051004,5975,5,N,4,0,263,93775,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1622153,6411,3,N,4,0,339,0,5865,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,5588,5,Y,8,0,296,73998,3426,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1846986,6215,5,Y,2,0,289,54828,3460,46,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
992751,6231,5,N,5,0,245,18405,5817,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **類別型 資料處理**

In [ ]:
df.head(2)

,acqic,contp,ecfg,etymd,fraud_ind,mcc,mchno,scity,stocn,stscd,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0,mcc_289,mcc_380,mcc_192,mcc_288,mcc_203,mcc_251,mcc_247,mcc_263,mcc_457,mcc_292,mchno_55807,mchno_59333,mchno_45597,mchno_67588,mchno_66634,mchno_86584,mchno_82174,mchno_78638,mchno_59369,mchno_44859,mchno_544,scity_6580,scity_3460,scity_6593,scity_1337,scity_5320,conam_3400
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,6881,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
483434,6881,5,N,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df['ecfg'] = (df['ecfg']=='Y').astype(int)

In [ ]:
df.head(2)

,acqic,contp,ecfg,etymd,fraud_ind,mcc,mchno,scity,stocn,stscd,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0,mcc_289,mcc_380,mcc_192,mcc_288,mcc_203,mcc_251,mcc_247,mcc_263,mcc_457,mcc_292,mchno_55807,mchno_59333,mchno_45597,mchno_67588,mchno_66634,mchno_86584,mchno_82174,mchno_78638,mchno_59369,mchno_44859,mchno_544,scity_6580,scity_3460,scity_6593,scity_1337,scity_5320,conam_3400
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,6881,5,0,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
483434,6881,5,0,0,0,457,59333,0,102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_disc = df[col_names_disc].copy()
df_disc.head()  # 處理前

,contp,etymd,stocn,stscd
txkey,,,,
516056,5,0,102,0
483434,5,0,102,0
1407164,5,5,102,0
1051004,5,4,102,0
1622153,3,4,102,0


In [ ]:
# 把'Y'變成1，'N'變成0
# df_disc['ecfg'] = (df_disc['ecfg']=='Y').astype(int)
# df_disc['flbmk'] = (df_disc['flbmk']=='Y').astype(int)
# df_disc['flg_3dsmk'] = (df_disc['flg_3dsmk']=='Y').astype(int)
# df_disc['insfg'] = (df_disc['insfg']=='Y').astype(int)
# df_disc['ovrlt'] = (df_disc['ovrlt']=='Y').astype(int)
# df_disc['fraud_ind'] = (df_disc['fraud_ind']=='Y').astype(int)

In [ ]:
from sklearn.preprocessing import LabelEncoder
# 把每個類別 mapping 到某個整數，不會增加新欄位

les = {}
for c in col_names_disc:
    le = LabelEncoder()
    df_disc.loc[:,c] = le.fit_transform(df_disc.loc[:,c])
    les.update({c:le})
df_disc.head()
# print(les) {'contp': LabelEncoder(), 'csmcu': LabelEncoder() ... }

,contp,etymd,stocn,stscd
txkey,,,,
516056,5,0,98,0
483434,5,0,98,0
1407164,5,5,98,0
1051004,5,4,98,0
1622153,3,4,98,0


In [ ]:
from sklearn.preprocessing import OneHotEncoder
# OneHotEncoder為每個類別新增一個欄位，用 0/1 表示是否

ohe = OneHotEncoder(sparse=False)
X_disc = ohe.fit_transform(df_disc)
# cut_point = ohe.feature_indices_
# print("feature cut point: ", cut_point)
X_disc.shape, type(X_disc)

((1521787, 126), numpy.ndarray)

In [ ]:
new_col_names_disc = []

for c in col_names_disc: 
    le = les[c]  # 從上面的les dict 取出 LabelEncoder()
    new_col_names_disc += [ c+'_'+str(cl) for cl in le.classes_ ]  # 建立新的欄位
assert len(new_col_names_disc) == X_disc.shape[1]

In [ ]:
df_disc = pd.DataFrame(data=X_disc, index=df.index, columns=new_col_names_disc)
df_disc = df_disc.loc[:, df_disc.columns != 'fraud_ind_1']
df_disc.head()

,contp_0,contp_1,contp_2,contp_3,contp_4,contp_5,contp_6,etymd_0,etymd_1,etymd_2,etymd_3,etymd_4,etymd_5,etymd_6,etymd_7,etymd_8,etymd_9,etymd_10,stocn_0,stocn_1,stocn_2,stocn_3,stocn_4,stocn_5,stocn_6,stocn_7,stocn_8,stocn_9,stocn_10,stocn_11,stocn_12,stocn_13,stocn_14,stocn_15,stocn_16,stocn_17,stocn_18,stocn_19,stocn_20,stocn_22,...,stocn_72,stocn_73,stocn_74,stocn_75,stocn_76,stocn_77,stocn_78,stocn_79,stocn_80,stocn_81,stocn_82,stocn_83,stocn_84,stocn_85,stocn_86,stocn_87,stocn_88,stocn_89,stocn_90,stocn_91,stocn_92,stocn_93,stocn_94,stocn_95,stocn_96,stocn_97,stocn_98,stocn_99,stocn_100,stocn_101,stocn_102,stocn_104,stocn_105,stocn_106,stocn_107,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
483434,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1407164,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1051004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1622153,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
tmp = col_names_disc + manipulation
print(tmp)
df = df.drop(df.loc[ :,tmp ], axis = 1)
df

['contp', 'etymd', 'stocn', 'stscd', 'acqic', 'mcc', 'mchno', 'scity']


,ecfg,fraud_ind,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0,mcc_289,mcc_380,mcc_192,mcc_288,mcc_203,mcc_251,mcc_247,mcc_263,mcc_457,mcc_292,mchno_55807,mchno_59333,mchno_45597,mchno_67588,mchno_66634,mchno_86584,mchno_82174,mchno_78638,mchno_59369,mchno_44859,mchno_544,scity_6580,scity_3460,scity_6593,scity_1337,scity_5320,conam_3400
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
483434,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1407164,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1051004,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1622153,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1846986,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
992751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 結合 數值型 和 類別型(binary)的資料的資料

df = pd.concat((df, df_cont, df_disc), 1)
df.shape

(1521787, 174)

In [ ]:
df

,ecfg,fraud_ind,acqic_3348,acqic_3221,acqic_6215,acqic_6540,acqic_5720,acqic_6760,acqic_3207,acqic_6331,acqic_6769,acqic_6856,acqic_6716,acqic_5975,acqic_6881,acqic_6189,acqic_6767,acqic_6032,acqic_6231,acqic_0,mcc_289,mcc_380,mcc_192,mcc_288,mcc_203,mcc_251,mcc_247,mcc_263,mcc_457,mcc_292,mchno_55807,mchno_59333,mchno_45597,mchno_67588,mchno_66634,mchno_86584,mchno_82174,mchno_78638,mchno_59369,mchno_44859,...,stocn_72,stocn_73,stocn_74,stocn_75,stocn_76,stocn_77,stocn_78,stocn_79,stocn_80,stocn_81,stocn_82,stocn_83,stocn_84,stocn_85,stocn_86,stocn_87,stocn_88,stocn_89,stocn_90,stocn_91,stocn_92,stocn_93,stocn_94,stocn_95,stocn_96,stocn_97,stocn_98,stocn_99,stocn_100,stocn_101,stocn_102,stocn_104,stocn_105,stocn_106,stocn_107,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
483434,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1407164,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1051004,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1622153,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1846986,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
992751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
df['conam'].describe()

count    1.521787e+06
mean     3.432328e-15
std      1.000000e+00
min     -1.625396e+00
25%     -5.403618e-01
50%     -1.559600e-01
75%      3.788859e-01
max      1.627091e+01
Name: conam, dtype: float64

In [ ]:
df['conam_3400'].describe()

count    1.521787e+06
mean     3.541889e-04
std      1.881658e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: conam_3400, dtype: float64

# **存檔**

In [ ]:
!pip install --upgrade gupload

from google.colab import drive
drive.mount('drive')

     |████████████████████████████████| 61kB 2.6MB/s 
     |████████████████████████████████| 81kB 3.5MB/s 
ERROR: earthengine-api 0.1.266 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.7.10 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
  Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
Mounted at drive


In [ ]:
# df = pd.concat((df, y_train), 1)
train = df[:1141340]  #train.csv=1141340 / test.csv:380447
test = df[1141340:]

In [ ]:
df

,conam,contp_0,contp_1,contp_2,contp_3,contp_4,contp_5,contp_6,csmcu_0,csmcu_1,csmcu_2,csmcu_3,csmcu_4,csmcu_5,csmcu_6,csmcu_7,csmcu_9,csmcu_10,csmcu_11,csmcu_12,csmcu_13,csmcu_14,csmcu_15,csmcu_16,csmcu_17,csmcu_18,csmcu_19,csmcu_20,csmcu_21,csmcu_22,csmcu_23,csmcu_24,csmcu_25,csmcu_26,csmcu_27,csmcu_28,csmcu_29,csmcu_30,csmcu_31,csmcu_32,...,stocn_73,stocn_74,stocn_75,stocn_76,stocn_77,stocn_78,stocn_79,stocn_80,stocn_81,stocn_82,stocn_83,stocn_84,stocn_85,stocn_86,stocn_87,stocn_88,stocn_89,stocn_90,stocn_91,stocn_92,stocn_93,stocn_94,stocn_95,stocn_96,stocn_97,stocn_98,stocn_99,stocn_100,stocn_101,stocn_102,stocn_104,stocn_105,stocn_106,stocn_107,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4,fraud_ind_0
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,-0.349849,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
483434,-0.349849,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1407164,0.897173,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1051004,0.146318,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1622153,2.859350,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,-1.621970,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1846986,-0.112987,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
992751,-0.199504,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# df[1141340:]['fraud_ind_0'] = df[1141340:]['fraud_ind_0'].replace({0:1, 1:0})  # somehow the data is opposite

print('\n=========================================')
print('Rmk: mean==fraud ratio\n')

print( df[:1141340]['fraud_ind'].describe(), '\n' )
print( df[1141340:]['fraud_ind'].describe() )


Rmk: mean==fraud ratio

count    1.141340e+06
mean     1.339040e-02
std      1.149396e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: fraud_ind, dtype: float64 

count    380447.000000
mean          0.013332
std           0.114691
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: fraud_ind, dtype: float64


In [ ]:
# Write the DataFrame to CSV file.
with open('/content/drive/My Drive/FinTechFile/train_parsed_V2.csv', 'w') as f:
  df[:1141340].to_csv(f)

In [ ]:
# Write the DataFrame to CSV file.
with open('/content/drive/My Drive/FinTechFile/test_parsed_V2.csv', 'w') as f:
  df[1141340:].to_csv(f)

In [ ]:
df


,conam,contp_0,contp_1,contp_2,contp_3,contp_4,contp_5,contp_6,csmcu_0,csmcu_1,csmcu_2,csmcu_3,csmcu_4,csmcu_5,csmcu_6,csmcu_7,csmcu_9,csmcu_10,csmcu_11,csmcu_12,csmcu_13,csmcu_14,csmcu_15,csmcu_16,csmcu_17,csmcu_18,csmcu_19,csmcu_20,csmcu_21,csmcu_22,csmcu_23,csmcu_24,csmcu_25,csmcu_26,csmcu_27,csmcu_28,csmcu_29,csmcu_30,csmcu_31,csmcu_32,...,stocn_73,stocn_74,stocn_75,stocn_76,stocn_77,stocn_78,stocn_79,stocn_80,stocn_81,stocn_82,stocn_83,stocn_84,stocn_85,stocn_86,stocn_87,stocn_88,stocn_89,stocn_90,stocn_91,stocn_92,stocn_93,stocn_94,stocn_95,stocn_96,stocn_97,stocn_98,stocn_99,stocn_100,stocn_101,stocn_102,stocn_104,stocn_105,stocn_106,stocn_107,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4,fraud_ind_0
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,-0.349849,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
483434,-0.349849,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1407164,0.897173,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1051004,0.146318,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1622153,2.859350,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668463,-1.621970,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1846986,-0.112987,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
992751,-0.199504,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
